In [25]:
from langgraph.graph import StateGraph, START, END
from langchain_ollama import ChatOllama
from typing import TypedDict, Annotated
from pydantic import BaseModel, Field
import operator

In [26]:
# model creation
model = ChatOllama(model="gemma3:4b")

In [27]:
# output schema
class EvaluateSchema(BaseModel):
    feedback: str = Field(description="Detailed feedback for essay")
    score: int = Field(description="Score out of 10", ge=0, le=10)

In [28]:
# example essay
essay = """
THE "Laptop Stickers" COLLEGE ESSAY EXAMPLE
My laptop is like a passport. It is plastered with stickers all over the outside, inside, and bottom. Each sticker is a stamp, representing a place I've been, a passion I've pursued, or community I've belonged to. These stickers make for an untraditional first impression at a meeting or presentation, but it's one I'm proud of. Let me take you on a quick tour:

"We < 3 Design," bottom left corner. Art has been a constant for me for as long as I can remember. Today my primary engagement with art is through design. I've spent entire weekends designing websites and social media graphics for my companies. Design means more to me than just branding and marketing; it gives me the opportunity to experiment with texture, perspective, and contrast, helping me refine my professional style.

"Common Threads," bottom right corner. A rectangular black and red sticker displaying the theme of the 2017 TEDxYouth@Austin event. For years I've been interested in the street artists and musicians in downtown Austin who are so unapologetically themselves. As a result, I've become more open-minded and appreciative of unconventional lifestyles. TED gives me the opportunity to help other youth understand new perspectives, by exposing them to the diversity of Austin where culture is created, not just consumed.

Poop emoji, middle right. My 13-year-old brother often sends his messages with the poop emoji 'echo effect,' so whenever I open a new message from him, hundreds of poops elegantly cascade across my screen. He brings out my goofy side, but also helps me think rationally when I am overwhelmed. We don't have the typical "I hate you, don't talk to me" siblinghood (although occasionally it would be nice to get away from him); we're each other's best friends. Or at least he's mine.

"Lol ur not Harry Styles," upper left corner. Bought in seventh grade and transferred from my old laptop, this sticker is torn but persevering with layers of tape. Despite conveying my fangirl-y infatuation with Harry Styles' boyband, One Direction, for me Styles embodies an artist-activist who uses his privilege for the betterment of society. As a $42K donor to the Time's Up Legal Defense Fund, a hair donor to the Little Princess Trust, and promoter of LGBTQ+ equality, he has motivated me to be a more public activist instead of internalizing my beliefs.

"Catapult," middle right. This is the logo of a startup incubator where I launched my first company, Threading Twine. I learned that business can provide others access to fundamental human needs, such as economic empowerment of minorities and education. In my career, I hope to be a corporate advocate for the empowerment of women, creating large-scale impact and deconstructing institutional boundaries that obstruct women from working in high-level positions. Working as a women's rights activist will allow me to engage in creating lasting movements for equality, rather than contributing to a cycle that elevates the stances of wealthy individuals.

"Thank God it's Monday," sneakily nestled in the upper right corner. Although I attempt to love all my stickers equally (haha), this is one of my favorites. I always want my association with work to be positive.

And there are many others, including the horizontal, yellow stripes of the Human Rights Campaign; "The Team," a sticker from the Model G20 Economics Summit where I collaborated with youth from around the globe; and stickers from "Kode with Klossy," a community of girls working to promote women's involvement in underrepresented fields.

When my computer dies (hopefully not for another few years), it will be like my passport expiring. It'll be difficult leaving these moments and memories behind, but I probably won't want these stickers in my 20s anyways (except Harry Styles, that's never leaving). My next set of stickers will reveal my next set of aspirations. They hold the key to future paths I will navigate, knowledge I will gain, and connections I will make.
"""

In [29]:
# model + schema
structured_model = model.with_structured_output(schema=EvaluateSchema)

In [30]:
prompt = f"Evaluate the language quality of the following essay and assign a score out of 10 \n {essay}"
output = structured_model.invoke(prompt)

### Outputs from the structured-model

In [31]:
output.feedback

"This is a fascinating and creative essay, but its language quality needs significant improvement to truly shine for a college application. Here's a breakdown and a score:"

In [32]:
output.score

6

In [46]:
# create State
class UPSCState(TypedDict):
    essay: str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str
    individual_scores: Annotated[list[int], operator.add]
    avg_score: float

### Defining Nodes

In [47]:
def evaluate_language(state: UPSCState) -> UPSCState:

    prompt = f"Evaluate the language quality of the following essay and assign a score out of 10 \n {state['essay']}"
    output = structured_model.invoke(prompt)

    return {"language_feedback": output.feedback, 'individual_scores': [output.score]}

In [48]:
def evaluate_analysis(state: UPSCState) -> UPSCState:
    prompt = f"Evaluate the depth of the analysis of the following essay and assign a score out of 10 \n {state['essay']}"
    output = structured_model.invoke(prompt)

    return {"analysis_feedback": output.feedback, 'individual_scores': [output.score]}

In [49]:
def evaluate_thought(state: UPSCState) -> UPSCState:
    prompt = f"Evaluate the clarity of thought of the following essay and assign a score out of 10 \n {state['essay']}"
    output = structured_model.invoke(prompt)

    return {"clarity_feedback": output.feedback, 'individual_scores': [output.score]}

In [50]:
def final_evaluation(state: UPSCState) -> UPSCState:
    
    # summary feedback
    prompt = f"Based on the following feedbacks create a summarized feedback \n language feedback - {state['language_feedback']} \n depth of analysis feedback - {state['analysis_feedback']} \n clarity of thought feedback - {state['clarity_feedback']}"
    overall_feedback = model.invoke(prompt).content

    # average calculate
    avg_score = sum(state['individual_scores'])/len(state['individual_scores'])

    return {"overall_feedback": overall_feedback, "avg_score": avg_score}

In [51]:
# creating graph
graph = StateGraph(UPSCState)

# nodes
graph.add_node("evaluate_language", evaluate_language)
graph.add_node("evaluate_analysis", evaluate_analysis)
graph.add_node("evaluate_thought", evaluate_thought)
graph.add_node("final_evaluation", final_evaluation)


# edges
graph.add_edge(START, "evaluate_language")
graph.add_edge(START, "evaluate_analysis")
graph.add_edge(START, "evaluate_thought")

graph.add_edge("evaluate_language", "final_evaluation")
graph.add_edge("evaluate_analysis", "final_evaluation")
graph.add_edge("evaluate_thought", "final_evaluation")

graph.add_edge("final_evaluation", END)


# compile graph
workflow = graph.compile()

In [52]:
# to see graph diagram
workflow.get_graph().print_ascii()

                                    +-----------+                                     
                                   *| __start__ |**                                   
                              ***** +-----------+  *****                              
                        ******            *             ******                        
                   *****                  *                   *****                   
                ***                       *                        ***                
+-------------------+           +-------------------+           +------------------+  
| evaluate_analysis |           | evaluate_language |           | evaluate_thought |  
+-------------------+***        +-------------------+         **+------------------+  
                        ******            *             ******                        
                              *****       *        *****                              
                                   ***    *

In [53]:
initial_state = {
    "essay": essay
}
final_state = workflow.invoke(initial_state)

In [54]:
final_state

{'essay': '\nTHE "Laptop Stickers" COLLEGE ESSAY EXAMPLE\nMy laptop is like a passport. It is plastered with stickers all over the outside, inside, and bottom. Each sticker is a stamp, representing a place I\'ve been, a passion I\'ve pursued, or community I\'ve belonged to. These stickers make for an untraditional first impression at a meeting or presentation, but it\'s one I\'m proud of. Let me take you on a quick tour:\n\n"We < 3 Design," bottom left corner. Art has been a constant for me for as long as I can remember. Today my primary engagement with art is through design. I\'ve spent entire weekends designing websites and social media graphics for my companies. Design means more to me than just branding and marketing; it gives me the opportunity to experiment with texture, perspective, and contrast, helping me refine my professional style.\n\n"Common Threads," bottom right corner. A rectangular black and red sticker displaying the theme of the 2017 TEDxYouth@Austin event. For years